# 基于条件随机场模型的中文分词算法

[![GitHub license](https://img.shields.io/github/license/Dragon1573/Revision-3A?style=for-the-badge)](https://github.com/Dragon1573/Revision-3A/blob/master/LICENSE)

## 准备语料库

准备一个已经分好词的、规模足够庞大的语料库，用于条件随机场模型的参数学习。语料库由大量类似下面的句子组成，词语之间采用空格符分隔。

> 尽管 印尼 中央 和 地方政府 已 派出 上千人 的 灭火队，但 由于 该 地区 长期 干旱 少 雨，所以 火势 至今 未 得到 有效 控制。

## 语料库特征初步学习

需要在语料库的每个词组中分析出每个字的状态，如“收益”需要改为“收\B 益\E”。对语料库中每个分好的词添加状态信息，标记后的语句如下：

> 尽\B 管\E 印\B 尼\E 中\B 央\E 和\S 地\S 方\M 政\M 府\E 已\S 派\B 出\E 上\B 千\M 人\E 的\S 灭\B 火\M 队\E ，\S 但\S 由\B 于\E 该\S 地\B 区\E 长\B 期\E 干\B 旱\E 少\S 雨\S ，\S 所\B 以\E 火\B 势\E 至\B 今\E 未\S 得\B 到\E 有\B 效\E 控\B 制\E 。\S

## 词语特征学习

词语特征学习是分词过程中非常重要的一步。

### 导入程序包，并定义全局变量

In [1]:
import json
import datetime

LABEL = ('B', 'M', 'E', 'S')

### 输入需要分词的字符串

In [2]:
string = input('请输入目标语句：')
start = datetime.datetime.now()

请输入目标语句： 明天去上学


### 语料库预处理

语料库以标点符号区分每个句子，而在程序中，标点符号处不被认为是一个句子的开始/结束。所以我们需要对语料库执行预处理，将标点符号转换为起止标记。

In [3]:
with open('msr_training.utf8.ic', 'r', encoding='UTF-8') as database:
    document = database.readlines()
for line in range(len(document)):
    if ord(document[line][0]) < 0x4E00 or ord(document[line][0]) > 0x9FA5:
        document[line] = '^' + document[line][1:]
if document[0][0] != '^':
    document.insert(0, '^|S\n')
if document[-1][0] != '^':
    document.append('^|S\n')

### 统计各种字符在各种状态下出现的次数

In [4]:
R = {item: {label: 0 for label in ('Count', 'B', 'M', 'E', 'S')} for item in list(string)}
for line in document:
    if line[0] in R.keys():
        R[line[0]]['Count'] += 1
        R[line[0]][line[-2]] += 1

### 统计各种字符在各种状态下出现的概率

In [5]:
for key in R.keys():
    for label in LABEL:
        try:
            R[key][label] /= R[key]['Count']
        except ZeroDivisionError:
            pass

### 计算每个字转移到下一个字的状态概率

In [6]:
P = {word: {label_A: {label_B: 0 for label_B in LABEL} for label_A in LABEL} for word in string}
for k in range(1, len(document) - 1):
    if document[k][0] in P.keys():
        P[document[k][0]][document[k][-2]][document[k + 1][-2]] += 1
for word in P.keys():
    for label_A in P[word].keys():
        for label_B in P[word][label_A].keys():
            try:
                P[word][label_A][label_B] = P[word][label_A][label_B] / sum(P[word][label_A].values())
            except ZeroDivisionError:
                # 将其统一定义为0即可
                P[word][label_A][label_B] = 0

### 计算词组复现概率

#### 向前组合

In [7]:
W_prev = {
    string[k]: {state_A: {'All': 0} for state_A in LABEL}
    for k in range(len(string))
}
temp = '^' + string + '^'
for line in range(1, len(document) - 1):
    for word in range(1, len(temp) - 1):
        if document[line][0] == temp[word]:
            W_prev[document[line][0]][document[line][-2]]['All'] += 1
            if document[line - 1][0] == temp[word - 1]:
                W_prev[document[line][0]][document[line][-2]].setdefault(document[line - 1][0], 0)
                W_prev[document[line][0]][document[line][-2]][document[line - 1][0]] += 1
for word in range(1, len(temp) - 1):
    for state in W_prev[temp[word]].keys():
        W_prev[temp[word]][state].setdefault(temp[word - 1], 0)
        try:
            W_prev[temp[word]][state][temp[word - 1]] = (
                W_prev[temp[word]][state][temp[word - 1]]
                / W_prev[temp[word]][state]['All']
            )
        except ZeroDivisionError:
            W_prev[temp[word]][state][temp[word - 1]] = 0

#### 向后组合

In [8]:
W_subs = {string[k]: {state_A: {'All': 0} for state_A in LABEL} for k in range(len(string))}
temp = '^' + string + '^'
for line in range(1, len(document) - 1):
    for word in range(1, len(temp) - 1):
        if document[line][0] == temp[word]:
            W_subs[document[line][0]][document[line][-2]]['All'] += 1
            if document[line + 1][0] == temp[word + 1]:
                W_subs[document[line][0]][document[line][-2]].setdefault(document[line + 1][0], 0)
                W_subs[document[line][0]][document[line][-2]][document[line + 1][0]] += 1
for word in range(1, len(temp) - 1):
    for state in W_subs[temp[word]].keys():
        W_subs[temp[word]][state].setdefault(temp[word + 1], 0)
        try:
            W_subs[temp[word]][state][temp[word + 1]] = (
                W_subs[temp[word]][state][temp[word + 1]] / W_subs[temp[word]][state]['All']
            )
        except ZeroDivisionError:
            W_subs[temp[word]][state][temp[word + 1]] = 0

## 开始分词

### 输入语句并切分为语素

$4050472$行数据对于`Python3`来说实在过于庞大，如果对语料库整体进行处理，每一步产生的数据量都是爆炸式的。所以，我们将目标语句的输入放到特征训练之前，对目标语句执行针对性训练。这样可以剔除分词时不会被使用的数据，极大程度地压缩训练过程中产生的数据量。

### 初始化字与状态的初始矩阵映射关系表

In [9]:
S = [{state: {'Rate': 0, 'Path': 'B'} for state in LABEL} for word in string]

### 计算字与状态对应关系

In [10]:
temp = '^' + string + '^'
for state in LABEL:
    S[0][state]['Rate'] = (
        W_prev[string[0]][state]['^'] + W_subs[string[0]][state][string[1]] + R[string[0]][state]
    )
for word in range(2, len(temp) - 1):
    for state_A in LABEL:
        routes = [
            P[temp[word - 1]][state_B][state_A] * S[word - 2][state_B]['Rate']
            for state_B in LABEL
        ]
        maximum = max(routes)
        S[word - 1][state_A]['Path'] = LABEL[routes.index(maximum)]
        S[word - 1][state_A]['Rate'] = (
            maximum + W_prev[temp[word]][state_A][temp[word - 1]]
            + W_subs[temp[word]][state_A][temp[word + 1]] + R[temp[word]][state_A]
        )

### 获得分词结果

In [11]:
result = []
temp = [rate['Rate'] for rate in S[-1].values()]
index = temp.index(max(temp))
current_state = tuple(S[-1].keys())[index]
result.append('\\' + current_state + ' ')
for word in range(len(string) - 1, -1, -1):
    result.append(string[word])
    result.append('\\' + S[word][current_state]['Path'] + ' ')
    current_state = S[word][current_state]['Path']
result.pop()
result.reverse()
result = ''.join(result)
print('分词标签：', result)
print('分词结果：', result.replace(r'\B ', '').replace(r'\M ', '').replace(r'\E ', ' ').replace(r'\S ', ' '))
end = datetime.datetime.now()
print('累计用时：', end - start)

分词标签： 明\B 天\E 去\S 上\S 学\S 
分词结果： 明天 去 上 学 
累计用时： 0:00:17.970923
